# 전국의 치킨집 정보 표시 : 부어치킨

# module import
requests, selenium, BeautifulSoup : 크롤링 모듈
folium : 지도 모듈

In [152]:
import requests
import folium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time 
import json

# 테스트 코드 작성(1페이지)

In [24]:
# 부어치킨 매장찾기 페이지
url = 'http://www.boor.co.kr/store/store.aspx' 
store_type = '%ub9e4%uc7a5%ucc3e%uae30'
requests.utils.unquote(store_type)
params = {
    'store_type': store_type
}
resp = requests.get(url, params=params)
soup = bs(resp.text, 'html.parser')
jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
store_list = [(j, a, t) for j, a, t in zip(jijum_list, tel_list, addr_list )]

[('부어치킨[가능점]', '031-877-9934', '경기도 의정부시 가능동 732-21'),
 ('부어치킨[가좌1동점]', '032-571-8417', '인천광역시 서구 가좌동 142-36 부어치킨'),
 ('부어치킨[가좌4동점]', '032-583-6699', '인천광역시 서구 가좌동 350-5'),
 ('부어치킨[가좌동점]', '032-576-0709', '인천광역시 서구 가좌동 30-25'),
 ('부어치킨[간석1동점]', '032-425-2727', '인천광역시 남동구 간석1동 285-7 오성상가 104호 부어치킨'),
 ('부어치킨[갈마점_대전지사]', '042-525-9225', '대전광역시 서구 갈마동 351-2 부어치킨')]

# 모든 페이지의 정보 크롤링
## 정보
### 대상 url
페이지가 바뀔 때 get방식으로 페이지가 드러나지 않음
-> requests를 사용할 수 없으므로 selenium으로 대체
### selenium
selenium의 headless 모드를 사용하면 크롤링을 제대로 못하는 현상 발생
해결방법을 찾지 못하여 headless 모드 없이 창 켜진채로 크롤링 수행

In [153]:
# # headless options
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')

chrome_driver = 'D:/bigdata/Python_workspace/jupyter/chromedriver.exe'
# # headless driver
# driver = webdriver.Chrome(chrome_driver, chrome_options=options)
driver = webdriver.Chrome(chrome_driver)
url = 'http://www.boor.co.kr/store/store.aspx?store_type=%ub9e4%uc7a5%ucc3e%uae30'
# 1페이지 불러오기
driver.get(url)
# 10페이지 단위를 세는 카운터
cnt = 0
# 가게 정보를 담을 리스트
store_list = []
while True:
    # 콘솔창에 크롤링 진행상황 표시(*1페이지)
    print(cnt*10+1,'page')
    # 페이지를 BeautifulSoup으로 변환
    resp = driver.page_source
    soup = bs(resp, 'html.parser')
    # 데이터 가공 : jijum, tel, addr
    jijum_list = list(map(lambda i:i.text, soup.select('li.sto_title')))
    addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
    tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
    # 가공한 데이터를 store_list에 저장
    store_list.extend([{'jijum': j, 'tel': a, 'addr': t} for j, a, t in zip(jijum_list, tel_list, addr_list )])
    # 크롤링이 제대로 수행되도록 간격 설정
    time.sleep(1)
    for i in range(2,11):
        # 콘솔창에 크롤링 진행상황 표시(*2~*0 페이지)
        print(cnt*10+i,'page')
        # 진행하는 페이지가 있을 경우
        try:
            elem = driver.find_element_by_css_selector(f'div.PagingCell[onclick="JavaScript:Page({cnt*10 + i})"]')
            elem.click()
        # 진행하려는 페이지가 없을 경우 나오도록 조건 설정        
        except Exception:
            break
        # 페이지를 BeautifulSoup으로 변환
        resp = driver.page_source
        soup = bs(resp, 'html.parser')
        # 데이터 가공 : jijum, tel, addr
        jijum_list = list(map(lambda i:i.text.split('[')[1].split(']')[0], soup.select('li.sto_title')))
        addr_list = list(map(lambda i:i['addr'], soup.select('li.add_text[groupname=Address]')))
        tel_list = list(map(lambda i:i.text, soup.select('li.add_text[groupname=Tel]')))
        # 가공한 데이터를 store_list에 저장
        store_list.extend([{'jijum': j, 'tel': a, 'addr': t} for j, a, t in zip(jijum_list, tel_list, addr_list )])
        time.sleep(0.1)
    # 진행하는 페이지가 있을 경우
    try:
        elem = driver.find_element_by_css_selector('a.pageList[title="다음 10개"]')
        elem.click()
    # 진행하려는 페이지가 없을 경우 나오도록 조건 설정 
    except Exception:
        print('end')
        break
    # 페이지를 10페이지 단위로 추가
    cnt += 1
driver.close()

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
21 page
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
30 page
31 page
32 page
33 page
34 page
35 page
36 page
37 page
38 page
39 page
40 page
41 page
42 page
43 page
44 page
45 page
46 page
47 page
48 page
49 page
50 page
51 page
52 page
53 page
54 page
end


In [149]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        if fixed_addr == '':
            print('불러오지 못했습니다 : ', addr)
            return (0, 0)
        else:
            return getLatLng(fixed_addr)
        
def get_avg_point(x_list, y_list):
    x = sum(x_list) / len(x_list)
    y = sum(y_list) / len(y_list)
    return x, y

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    disabled_sup = list(set(df_noman['장애인편의발급기형태']))
    for x, y in addr_point_list:
        if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
            add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y, tel):
    marker = folium.Marker((x, y), popup=tel)
    icon=folium.Icon(color='orange',icon='glyphicon glyphicon-cutlery')
    icon.add_to(marker)
    return marker

In [131]:
for store in store_list:
    point = getLatLng(store['addr'])
    store['x'] = point[1]
    store['y'] = point[0]

In [132]:
with open('boorchicken.json','w',encoding='utf-8') as f:
    store_list_dump = json.dumps(store_list, indent=4, sort_keys=True, ensure_ascii=False)
    f.write(store_list_dump)

In [150]:
x_list = list(map(lambda store:float(store['x']),store_list))
y_list = list(map(lambda store:float(store['y']),store_list))
tel_list = list(map(lambda store:store['tel'],store_list))
map_boor = folium.Map(location=get_avg_point(x_list,y_list), zoom_start=7)
for x, y, tel in zip(x_list,y_list, tel_list):
    draw_marker(x, y, tel).add_to(map_boor)

In [151]:
map_boor